# Interpreting BERT Models (Part 1)

In [1]:
!pip install transformers
!pip install captum
!pip install seaborn

In this notebook we demonstrate how to interpret Bert models using  `Captum` library. In this particular case study we focus on a fine-tuned Question Answering model on SQUAD dataset using transformers library from Hugging Face: https://huggingface.co/transformers/

We show how to use interpretation hooks to examine and better understand embeddings, sub-embeddings, bert, and attention layers. 

Note: Before running this tutorial, please install `seaborn`, `pandas` and `matplotlib`, `transformers`(from hugging face) python packages.

In [2]:
import os
import sys

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

from transformers import BertTokenizer,BartTokenizer, BertForQuestionAnswering, BertConfig, BertForSequenceClassification, BartForSequenceClassification

from captum.attr import visualization as viz
from captum.attr import IntegratedGradients, LayerConductance, LayerIntegratedGradients
from captum.attr import configure_interpretable_embedding_layer, remove_interpretable_embedding_layer

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

The first step is to fine-tune BERT model on SQUAD dataset. This can be easiy accomplished by following the steps described in hugging face's official web site: https://github.com/huggingface/transformers#run_squadpy-fine-tuning-on-squad-for-question-answering 

Note that the fine-tuning is done on a `bert-base-uncased` pre-trained model.

After we pretrain the model, we can load the tokenizer and pre-trained BERT model using the commands described below. 

In [112]:
import os
os.path.isdir("../checkpoints/exp_bart_l_disc_model")

True

In [41]:
# pwd

In [113]:
# replace <PATH-TO-SAVED-MODEL> with the real path of the saved model
# model_path = 'bert-base-uncased'
model_path = 'facebook/bart-large'

# load model
model = BartForSequenceClassification.from_pretrained("../checkpoints/exp_bart_l_disc_model", return_dict=False)
model.to(device)
model.eval()
model.zero_grad()

# load tokenizer
tokenizer = BartTokenizer.from_pretrained(model_path)

hyp_path = "../checkpoints/exp_bart_b_hyp_mnli_model"
if hyp_path is not None:
    hyp_model = BartForSequenceClassification.from_pretrained(hyp_path, return_dict=False)
    hyp_tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')

A helper function to perform forward pass of the model and make predictions.

In [114]:
def predict(inputs, token_type_ids=None, position_ids=None, attention_mask=None):
    return model(inputs, 
#                  token_type_ids=token_type_ids,
#                  position_ids=position_ids, 
                 attention_mask=attention_mask, )

Defining a custom forward function that will allow us to access the start and end postitions of our prediction using `position` input argument.

In [115]:
def squad_pos_forward_func(inputs, token_type_ids=None, position_ids=None, attention_mask=None, position=0):
    pred = predict(inputs,
                   token_type_ids=token_type_ids,
                   position_ids=position_ids,
                   attention_mask=attention_mask)
    pred = pred[position]
    return pred

Let's compute attributions with respect to the `BertEmbeddings` layer.

To do so, we need to define baselines / references, numericalize both the baselines and the inputs. We will define helper functions to achieve that.

The cell below defines numericalized special tokens that will be later used for constructing inputs and corresponding baselines/references.

In [116]:
ref_token_id = tokenizer.pad_token_id # A token used for generating token reference
sep_token_id = tokenizer.sep_token_id # A token used as a separator between question and text and it is also added to the end of the text.
cls_token_id = tokenizer.cls_token_id # A token used for prepending to the concatenated question-text word sequence
print(ref_token_id)
print(sep_token_id)
print(cls_token_id)

1
2
0


Below we define a set of helper function for constructing references / baselines for word tokens, token types and position ids. We also provide separate helper functions that allow to construct the sub-embeddings and corresponding baselines / references for all sub-embeddings of `BertEmbeddings` layer.

In [117]:
def construct_input_ref_pair(premise, hypothesis, ref_token_id, sep_token_id, cls_token_id):
    premise_ids = tokenizer.encode(premise, add_special_tokens=False)
    hypothesis_ids = tokenizer.encode(hypothesis, add_special_tokens=False)

    # construct input token ids
    input_ids = [cls_token_id] + premise_ids + [sep_token_id] + [cls_token_id] + hypothesis_ids + [sep_token_id]

    # construct reference token ids 
    ref_input_ids = [cls_token_id] + [ref_token_id] * len(premise_ids) + [sep_token_id] + [cls_token_id] + \
        [ref_token_id] * len(hypothesis_ids) + [sep_token_id]

    return torch.tensor([input_ids], device=device), torch.tensor([ref_input_ids], device=device), len(premise_ids)

def construct_input_ref_token_type_pair(input_ids, sep_ind=0):
    seq_len = input_ids.size(1)
    token_type_ids = torch.tensor([[0 if i <= sep_ind else 1 for i in range(seq_len)]], device=device)
    ref_token_type_ids = torch.zeros_like(token_type_ids, device=device)# * -1
    return token_type_ids, ref_token_type_ids

def construct_input_ref_pos_id_pair(input_ids):
    seq_length = input_ids.size(1)
    position_ids = torch.arange(seq_length, dtype=torch.long, device=device)
    # we could potentially also use random permutation with `torch.randperm(seq_length, device=device)`
    ref_position_ids = torch.zeros(seq_length, dtype=torch.long, device=device)

    position_ids = position_ids.unsqueeze(0).expand_as(input_ids)
    ref_position_ids = ref_position_ids.unsqueeze(0).expand_as(input_ids)
    return position_ids, ref_position_ids
    
def construct_attention_mask(input_ids):
    return torch.ones_like(input_ids)

def construct_bert_sub_embedding(input_ids, ref_input_ids,
                                   token_type_ids, ref_token_type_ids,
                                   position_ids, ref_position_ids):
    input_embeddings = interpretable_embedding1.indices_to_embeddings(input_ids)
    ref_input_embeddings = interpretable_embedding1.indices_to_embeddings(ref_input_ids)

    input_embeddings_token_type = interpretable_embedding2.indices_to_embeddings(token_type_ids)
    ref_input_embeddings_token_type = interpretable_embedding2.indices_to_embeddings(ref_token_type_ids)

    input_embeddings_position_ids = interpretable_embedding3.indices_to_embeddings(position_ids)
    ref_input_embeddings_position_ids = interpretable_embedding3.indices_to_embeddings(ref_position_ids)
    
    return (input_embeddings, ref_input_embeddings), \
           (input_embeddings_token_type, ref_input_embeddings_token_type), \
           (input_embeddings_position_ids, ref_input_embeddings_position_ids)
    
def construct_whole_bert_embeddings(input_ids, ref_input_ids, \
                                    token_type_ids=None, ref_token_type_ids=None, \
                                    position_ids=None, ref_position_ids=None):
    input_embeddings = interpretable_embedding.indices_to_embeddings(input_ids, token_type_ids=token_type_ids, position_ids=position_ids)
    ref_input_embeddings = interpretable_embedding.indices_to_embeddings(ref_input_ids, token_type_ids=token_type_ids, position_ids=position_ids)
    
    return input_embeddings, ref_input_embeddings


Let's define the `question - text` pair that we'd like to use as an input for our Bert model and interpret what the model was forcusing on when predicting an answer to the question from given input text 

In [118]:
# premise, hypothesis = "The slang of one military generation passes on to the next , so the Marines who called the Koreans gooks in the '50s and the Vietnamese gooks in the '60s and '70s were the linguistic heirs of the Marines who called the Nicaraguans gooks in 1912 .","New military slang terms were created in the 1980s ."
premise, hypothesis = "a black man in a white uniform makes a spectacular reverse slam dunk to the crowd ' s amazement .","the man is asian"

Let's numericalize the question, the input text and generate corresponding baselines / references for all three sub-embeddings (word, token type and position embeddings) types using our helper functions defined above.

In [119]:
input_ids, ref_input_ids, sep_id = construct_input_ref_pair(premise, hypothesis, ref_token_id, sep_token_id, cls_token_id)
token_type_ids, ref_token_type_ids = construct_input_ref_token_type_pair(input_ids, sep_id)
position_ids, ref_position_ids = construct_input_ref_pos_id_pair(input_ids)
attention_mask = construct_attention_mask(input_ids)

indices = input_ids[0].detach().tolist()
all_tokens = tokenizer.convert_ids_to_tokens(indices)
print(tokenizer.decode(indices))
print(all_tokens)

<s>a black man in a white uniform makes a spectacular reverse slam dunk to the crowd's amazement.</s><s>the man is asian</s>
['<s>', 'a', 'Ġblack', 'Ġman', 'Ġin', 'Ġa', 'Ġwhite', 'Ġuniform', 'Ġmakes', 'Ġa', 'Ġspectacular', 'Ġreverse', 'Ġslam', 'Ġdunk', 'Ġto', 'Ġthe', 'Ġcrowd', "Ġ'", 'Ġs', 'Ġamaz', 'ement', 'Ġ.', '</s>', '<s>', 'the', 'Ġman', 'Ġis', 'Ġas', 'ian', '</s>']


Also, let's define the ground truth for prediction's start and end positions.

In [120]:

ground_truth_end_ind = "contradiction"
labels = ["contradiction","entailment","neutral"]

Now let's make predictions using input, token type, position id and a default attention mask.

In [121]:
scores = predict(input_ids, \
                                   token_type_ids=token_type_ids, \
                                   position_ids=position_ids, \
                                   attention_mask=attention_mask)[0]

#import pdb; pdb.set_trace()
print('Predicted Answer: ', labels[torch.argmax(scores)])

Predicted Answer:  contradiction


In [122]:
disc_probs = torch.nn.Softmax(dim=1)(scores)
disc_probs

tensor([[0.9405, 0.0025, 0.0570]], device='cuda:0', grad_fn=<SoftmaxBackward>)

There are two different ways of computing the attributions for `BertEmbeddings` layer. One option is to use `LayerIntegratedGradients` and compute the attributions with respect to that layer. The second option is to pre-compute the embeddings and wrap the actual embeddings with `InterpretableEmbeddingBase`. The pre-computation of embeddings for the second option is necessary because integrated gradients scales the inputs and that won't be meaningful on the level of word / token indices.

Since using `LayerIntegratedGradients` is simpler, let's use it here.

In [123]:
def custom_forward(inputs, token_type_ids=None, position_ids=None, attention_mask=None, position=0, true_label=None):
  outputs = predict(inputs, token_type_ids=token_type_ids, position_ids=position_ids, attention_mask=attention_mask)
  preds = outputs[0]
  if true_label is not None:
    return preds[:,true_label]
#   import pdb; pdb.set_trace()
  return preds.max(1).values

In [136]:
# lig = LayerIntegratedGradients(custom_forward, model.bert.embeddings)
lig = LayerIntegratedGradients(custom_forward, model.model.encoder.embed_tokens)

attributions, delta = lig.attribute(inputs=input_ids,
                                  baselines=ref_input_ids,
                                  additional_forward_args=(token_type_ids, position_ids, attention_mask, 0, 0), # revise this
                                  return_convergence_delta=True,
                                  n_steps=100, 
                                   )

A helper function to summarize attributions for each word token in the sequence.

In [137]:
def summarize_attributions(attributions):
    attributions = attributions.sum(dim=-1).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    return attributions

In [138]:
attributions_sum = summarize_attributions(attributions)

In [139]:
# storing couple samples in an array for visualization purposes
start_position_vis = viz.VisualizationDataRecord(
                        attributions_sum[1:],
                        torch.max(torch.softmax(scores[0], dim=0)),
#                         torch.max(scores),
                        torch.argmax(scores),
                        torch.argmax(scores),
                        str(ground_truth_end_ind),
                        attributions_sum.sum(),       
                        all_tokens[:-1],
                        delta)

# end_position_vis = viz.VisualizationDataRecord(
#                         attributions_end_sum,
#                         torch.max(torch.softmax(end_scores[0], dim=0)),
#                         torch.argmax(end_scores),
#                         torch.argmax(end_scores),
#                         str(ground_truth_end_ind),
#                         attributions_end_sum.sum(),       
#                         all_tokens,
#                         delta_end)

print('\033[1m', 'Visualizations For Start Position', '\033[0m')
viz.visualize_text([start_position_vis])

# print('\033[1m', 'Visualizations For End Position', '\033[0m')
# viz.visualize_text([end_position_vis])

 Visualizations For Start Position 


In [140]:
attributions_sum[1:]
# from IPython.display import Image
# Image(filename='img/bert/visuals_of_start_end_predictions.png')

tensor([ 0.0000,  0.0097,  0.3984, -0.0277,  0.0831,  0.0592, -0.0327, -0.0141,
        -0.0457,  0.0157,  0.0111,  0.0337, -0.0034,  0.0413,  0.0517,  0.0929,
         0.0090,  0.0887,  0.0723,  0.0718,  0.0402,  0.1057,  0.0000,  0.0000,
         0.0070, -0.0266, -0.0886,  0.7116,  0.5158], device='cuda:0',
       dtype=torch.float64)

In [141]:
delta

tensor([0.6964], device='cuda:0', dtype=torch.float64)

In [24]:
# hyp_ids = input_ids[:,(input_ids==102).nonzero()[0][-1]:].clone()
# hyp_ids[:,0]=101
hyp_ids = hyp_tokenizer(hypothesis,return_tensors='pt').input_ids
hyp_ids
hyp_out = hyp_model(input_ids=hyp_ids)

hyp_probs = torch.nn.Softmax(dim=1)(hyp_out[0])
hyp_probs
# input_ids

tensor([[0.0305, 0.0626, 0.9069]], grad_fn=<SoftmaxBackward>)

In [25]:
torch.nn.Softmax(dim=1)(scores - hyp_out[0])

tensor([[0.3199, 0.4900, 0.1902]], grad_fn=<SoftmaxBackward>)

In [26]:
attributions_sum - delta

tensor([ 0.0072,  0.0287, -0.0297,  0.0531,  0.0400, -0.0112, -0.1456, -0.0807,
        -0.1761,  0.0338, -0.0786,  0.3472, -0.2863,  0.1580,  0.1728,  0.2208,
        -0.0726, -0.1448, -0.1144, -0.1135,  0.0854,  0.0072,  0.0444,  0.0506,
        -0.0553, -0.0180,  0.0566,  0.3803,  0.1235,  0.0213,  0.1254,  0.2230,
        -0.0369, -0.4246, -0.0642, -0.3684, -0.0264,  0.0072],
       dtype=torch.float64)

From the results above we can tell that for predicting start position our model is focusing more on the question side. More specifically on the tokens `what` and `important`. It has also slight focus on the token sequence `to us` in the text side.

In contrast to that, for predicting end position, our model focuses more on the text side and has relative high attribution on the last end position token `kinds`.

# Multi-Embedding attribution

Now let's look into the sub-embeddings of `BerEmbeddings` and try to understand the contributions and roles of each of them for both start and end predicted positions.

To do so, we'd need to place interpretation hooks in each three of them.

Note that we could perform attribution by using `LayerIntegratedGradients` as well but in that case we have to call attribute three times for each sub-layer since currently `LayerIntegratedGradients` takes only a layer at a time. In the future we plan to support multi-layer attribution and will be able to perform attribution by only calling attribute once. 

`configure_interpretable_embedding_layer` function will help us to place interpretation hooks on each sub-layer. It returns `InterpretableEmbeddingBase` layer for each sub-embedding and can be used to access the embedding vectors. 

Note that we need to remove InterpretableEmbeddingBase wrapper from our model using remove_interpretable_embedding_layer function after we finish interpretation.


In [40]:
# interpretable_embedding1 = configure_interpretable_embedding_layer(model, 'bert.embeddings.word_embeddings')
# interpretable_embedding2 = configure_interpretable_embedding_layer(model, 'bert.embeddings.token_type_embeddings')
# interpretable_embedding3 = configure_interpretable_embedding_layer(model, 'bert.embeddings.position_embeddings')
interpretable_embedding1 = configure_interpretable_embedding_layer(model, 'model.shared')
interpretable_embedding2 = interpretable_embedding1
interpretable_embedding3 = interpretable_embedding1

AssertionError: InterpretableEmbeddingBase has already been configured for layer model.shared

`BertEmbeddings` has three sub-embeddings, namely, `word_embeddings`, `token_type_embeddings` and `position_embeddings` and this time we would like to attribute to each of them independently.
`construct_bert_sub_embedding` helper function helps us to construct input embeddings and corresponding references in a separation.

In [28]:
(input_embed, ref_input_embed), (token_type_ids_embed, ref_token_type_ids_embed), (position_ids_embed, ref_position_ids_embed) = construct_bert_sub_embedding(input_ids, ref_input_ids, \
                                         token_type_ids=token_type_ids, ref_token_type_ids=ref_token_type_ids, \
                                         position_ids=position_ids, ref_position_ids=ref_position_ids)

Now let's create an instance of `IntegratedGradients` and compute the attributions with respect to all those embeddings both for the start and end positions and summarize them for each word token.

In [29]:
ig = IntegratedGradients(custom_forward)

attributions_start = ig.attribute(inputs=(input_embed, token_type_ids_embed, position_ids_embed),
                                  baselines=(ref_input_embed, ref_token_type_ids_embed, ref_position_ids_embed),
                                  additional_forward_args=(attention_mask, 0))

attributions_start_word = summarize_attributions(attributions_start[0])

attributions_start_token_type = summarize_attributions(attributions_start[1])

attributions_start_position = summarize_attributions(attributions_start[2])


ValueError: too many values to unpack (expected 2)

An auxilary function that will help us to compute topk attributions and corresponding indices

In [ ]:
def get_topk_attributed_tokens(attrs, k=5):
    values, indices = torch.topk(attrs, k)
    top_tokens = [all_tokens[idx] for idx in indices]
    return top_tokens, values, indices

Removing interpretation hooks from all layers after finishing attribution.

In [ ]:
remove_interpretable_embedding_layer(model, interpretable_embedding1)
remove_interpretable_embedding_layer(model, interpretable_embedding2)
remove_interpretable_embedding_layer(model, interpretable_embedding3)

Computing topk attributions for all sub-embeddings and placing them in pandas dataframes for better visualization.

In [ ]:
top_words_start, top_words_val_start, top_word_ind_start = get_topk_attributed_tokens(attributions_start_word)

top_token_type_start, top_token_type_val_start, top_token_type_ind_start = get_topk_attributed_tokens(attributions_start_token_type)

top_pos_start, top_pos_val_start, pos_ind_start = get_topk_attributed_tokens(attributions_start_position)

df_start = pd.DataFrame({'Word(Index), Attribution': ["{} ({}), {}".format(word, pos, round(val.item(),2)) for word, pos, val in zip(top_words_start, top_word_ind_start, top_words_val_start)],
                   'Token Type(Index), Attribution': ["{} ({}), {}".format(ttype, pos, round(val.item(),2)) for ttype, pos, val in zip(top_token_type_start, top_token_type_ind_start, top_words_val_start)],
                   'Position(Index), Attribution': ["{} ({}), {}".format(position, pos, round(val.item(),2)) for position, pos, val in zip(top_pos_start, pos_ind_start, top_pos_val_start)]})
df_start.style.apply(['cell_ids: False'])

['{}({})'.format(token, str(i)) for i, token in enumerate(all_tokens)]

Below we can see top 5 attribution results from all three embedding types in predicting start positions.

#### Top 5 attributed embeddings for start position

In [ ]:
df_start

Word embeddings help to focus more on the surrounding tokens of the predicted answer's start position to such as em, ##power and ,. It also has high attribution for the tokens in the question such as what and ?.

In contrast to to word embedding, token embedding type focuses more on the tokens in the text part such as important,em and start token to.

Position embedding also has high attribution score for the tokens surrounding to such as us and important. In addition to that, similar to word embedding we observe important tokens from the question.

We can perform similar analysis, and visualize top 5 attributed tokens for all three embedding types, also for the end position prediction.


#### Top 5 attributed embeddings for end position

In [ ]:
# df_end

It is interesting to observe high concentration of highly attributed tokens such as `of`, `kinds`, `support` and `##power` for end position prediction.

The token `kinds`, which is the correct predicted token appears to have high attribution score both according word and position embeddings.


# Interpreting Bert Layers

Now let's look into the layers of our network. More specifically we would like to look into the distribution of attribution scores for each token across all layers in Bert model and dive deeper into specific tokens.  
We do that using one of layer attribution algorithms, namely, layer conductance. However, we encourage you to try out and compare the results with other algorithms as well.


Let's configure `InterpretableEmbeddingsBase` again, in this case in order to interpret the layers of our model.

In [ ]:
interpretable_embedding = configure_interpretable_embedding_layer(model, 'bert.embeddings')

Let's iterate over all layers and compute the attributions for all tokens. In addition to that let's also choose a specific token that we would like to examine in detail, specified by an id `token_to_explain` and store related information in a separate array.


Note: Since below code is iterating over all layers it can take over 5 seconds. Please be patient!

In [ ]:
layer_attrs_start = []

# The token that we would like to examine separately.
token_to_explain = 9 # the index of the token that we would like to examine more thoroughly
layer_attrs_start_dist = []

input_embeddings, ref_input_embeddings = construct_whole_bert_embeddings(input_ids, ref_input_ids, \
                                         token_type_ids=token_type_ids, ref_token_type_ids=ref_token_type_ids, \
                                         position_ids=position_ids, ref_position_ids=ref_position_ids)

for i in range(model.config.num_hidden_layers):
    lc = LayerConductance(custom_forward, model.bert.encoder.layer[i])
    layer_attributions_start = lc.attribute(inputs=input_embeddings, baselines=ref_input_embeddings, additional_forward_args=(token_type_ids, position_ids,attention_mask, 0))[0]
#     layer_attributions_end = lc.attribute(inputs=input_embeddings, baselines=ref_input_embeddings, additional_forward_args=(token_type_ids, position_ids,attention_mask, 1))[0]
 
    layer_attrs_start.append(summarize_attributions(layer_attributions_start).cpu().detach().tolist())
#     layer_attrs_end.append(summarize_attributions(layer_attributions_end).cpu().detach().tolist())

    # storing attributions of the token id that we would like to examine in more detail in token_to_explain
    layer_attrs_start_dist.append(layer_attributions_start[0,token_to_explain].cpu().detach().tolist())
#     layer_attrs_end_dist.append(layer_attributions_end[0,token_to_explain,:].cpu().detach().tolist())


The plot below represents a heat map of attributions across all layers and tokens for the start position prediction. 
It is interesting to observe that the question word `what` gains increasingly high attribution from layer one to nine. In the last three layers that importance is slowly diminishing.  
In contrary to `what` token, many other tokens have negative or close to zero attribution in the first 6 layers. 

We start seeing slightly higher attribution in tokens `important`, `us` and `to`. Interestingly token `em` is also assigned high attribution score which is remarkably high the last three layers.
And lastly, our correctly predicted token `to` for the start position gains increasingly positive attribution has relatively high attribution especially in the last two layers.


In [ ]:
fig, ax = plt.subplots(figsize=(15,5))
xticklabels=all_tokens
yticklabels=list(range(1,13))
ax = sns.heatmap(np.array(layer_attrs_start), xticklabels=xticklabels, yticklabels=yticklabels, linewidth=0.2)
plt.xlabel('Tokens')
plt.ylabel('Layers')
plt.show()

Now let's examine the heat map of the attributions for the end position prediction. In the case of end position prediction we again observe high attribution scores for the token `what` in the last 11 layers.
The correctly predicted end token `kinds` has positive attribution across all layers and it is especially prominent in the last two layers.

In [ ]:
# fig, ax = plt.subplots(figsize=(15,5))

# xticklabels=all_tokens
# yticklabels=list(range(1,13))
# ax = sns.heatmap(np.array(layer_attrs_end), xticklabels=xticklabels, yticklabels=yticklabels, linewidth=0.2) #, annot=True
# plt.xlabel('Tokens')
# plt.ylabel('Layers')

# plt.show()

It is interesting to note that when we compare the heat maps of start and end position, overall the colors for start position prediction on the map have darker intensities. This implies that there are less tokens that attribute positively to the start position prediction and there are more tokens which are negative indicators or signals of start position prediction.

Now let's dig deeper into specific tokens and look into the distribution of attributions per layer for the token `kinds` in the start and end positions. The box plot diagram below shows the presence of outliers especially in the first four layers and in layer 8. We also observe that for start position prediction interquartile range slowly decreases as we go deeper into the layers and finally it is dimishing.



In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
ax = sns.boxplot(data=layer_attrs_start_dist)
plt.xlabel('Layers')
plt.ylabel('Attribution')
plt.show()

Now let's plot same distribution but for the prediction of the end position. Here attribution has larger positive values across all layers and the interquartile range doesn't change much when moving deeper into the layers.

In [ ]:
# fig, ax = plt.subplots(figsize=(20,10))
# ax = sns.boxplot(data=layer_attrs_end_dist)
# plt.xlabel('Layers')
# plt.ylabel('Attribution')
# plt.show()

Now, let's remove interpretation hooks, since we finished interpretation at this point

In [ ]:
remove_interpretable_embedding_layer(model, interpretable_embedding)

In addition to that we can also look into the distribution of attributions in each layer for any input token. This will help us to better understand and compare the distributional patterns of attributions across multiple layers. We can for example represent attributions as a probability density function (pdf) and compute the entropy of it in order to estimate the entropy of attributions in each layer. This can be easily computed using a histogram.

In [ ]:
def pdf_attr(attrs, bins=100):
    return np.histogram(attrs, bins=bins, density=True)[0]

In this particular case let's compute the pdf for the attributions at end positions `kinds`. We can however do it for all tokens.

We will compute and visualize the pdfs and entropies using Shannon's Entropy measure for each layer for token `kinds`.

In [ ]:
layer_attrs_start_pdf = map(lambda layer_attrs_start_dist: pdf_attr(layer_attrs_start_dist), layer_attrs_start_dist)
layer_attrs_start_pdf = np.array(list(layer_attrs_start_pdf))

# summing attribution along embedding diemension for each layer
# size: #layers
attr_sum = np.array(layer_attrs_start_dist).sum(-1)

# size: #layers
layer_attrs_start_pdf_norm = np.linalg.norm(layer_attrs_start_pdf, axis=-1, ord=1)

#size: #bins x #layers
layer_attrs_start_pdf = np.transpose(layer_attrs_start_pdf)

#size: #bins x #layers
layer_attrs_start_pdf = np.divide(layer_attrs_start_pdf, layer_attrs_start_pdf_norm, where=layer_attrs_start_pdf_norm!=0)

The plot below visualizes the probability mass function (pmf) of attributions for each layer for the end position token `kinds`. From the plot we can observe that the distributions are taking bell-curved shapes with different means and variances.
We can now use attribution pdfs to compute entropies in the next cell.

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
plt.plot(layer_attrs_start_pdf)
plt.xlabel('Bins')
plt.ylabel('Density')
plt.legend(['Layer '+ str(i) for i in range(1,13)])
plt.show()

Below we calculate and visualize attribution entropies based on Shannon entropy measure where the x-axis corresponds to the number of layers and the y-axis corresponds to the total attribution in that layer. The size of the circles for each (layer, total_attribution) pair correspond to the normalized entropy value at that point.

In this particular example, we observe that the entropy doesn't change much from layer to layer, however in a general case entropy can provide us an intuition about the distributional characteristics of attributions in each layer and can be useful especially when comparing it across multiple tokens.


In [ ]:
fig, ax = plt.subplots(figsize=(20,10))

# replacing 0s with 1s. np.log(1) = 0 and np.log(0) = -inf
layer_attrs_start_pdf[layer_attrs_start_pdf == 0] = 1
layer_attrs_start_pdf_log = np.log2(layer_attrs_start_pdf)

# size: #layers
entropies= -(layer_attrs_start_pdf * layer_attrs_start_pdf_log).sum(0)

plt.scatter(np.arange(len(attr_sum)), attr_sum, s=entropies * 100)
plt.xlabel('Layers')
plt.ylabel('Total Attribution')
plt.show()

In the Part 2 of this tutorial we will to go deeper into attention layers, heads and compare the attributions with the attention weight matrices, study and discuss related statistics.